In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma
import ipywidgets as widgets
from IPython.display import display

# --- 1. Define the Double Gamma Function (SPM/Glover style) ---
def double_gamma_hrf(t, lag=6, disp=1, u_lag=16, u_disp=1, ratio=1/6):
    """
    t: time vector
    lag: time to peak response
    disp: dispersion of response
    u_lag: time to undershoot
    u_disp: dispersion of undershoot
    ratio: ratio of undershoot to peak
    """
    # Gamma distribution pdf: (x**(a-1) * exp(-x/b)) / (gamma(a)*b**a)
    # SPM uses a slightly simplified version, but scipy.stats.gamma is accurate
    # peak_gamma: a=lag/disp, scale=disp
    
    # Positive (Main) Gamma
    pos = gamma.pdf(t, lag/disp, scale=disp)
    
    # Negative (Undershoot) Gamma
    neg = gamma.pdf(t, u_lag/u_disp, scale=u_disp)
    
    # Combine (Main - scaled Undershoot)
    hrf = pos - ratio * neg
    
    # Scale max to 1 for visualization
    hrf /= np.max(hrf) 
    pos /= np.max(pos) # visual scaling only
    neg /= np.max(neg) # visual scaling only
    
    return pos, neg, hrf

# --- 2. Interactive Plotter ---
def plot_hrf_components(show_pos, show_neg, show_sum):
    t = np.linspace(0, 32, 100)
    pos, neg, hrf = double_gamma_hrf(t)
    
    plt.figure(figsize=(10, 6))
    
    if show_pos:
        plt.plot(t, pos, 'g--', label='Positive Gamma (Oxygen Supply)', linewidth=2)
        plt.fill_between(t, pos, color='green', alpha=0.1)
        
    if show_neg:
        # We plot negative as negative values for intuition, though mathematically it's a subtraction
        plt.plot(t, -neg*0.16, 'r--', label='Negative Gamma (Undershoot)', linewidth=2) 
        plt.fill_between(t, -neg*0.16, color='red', alpha=0.1)

    if show_sum:
        plt.plot(t, hrf, 'k-', label='Resulting HRF (The "Model")', linewidth=4)
    
    plt.title("Deconstructing the BOLD Signal (Double Gamma)", fontsize=15)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude (a.u.)")
    plt.axhline(0, color='gray', linewidth=0.5)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.show()

# --- 3. Create Widgets ---
ui = widgets.HBox([
    widgets.Checkbox(value=True, description='Show Positive Gamma'),
    widgets.Checkbox(value=False, description='Show Negative Gamma'),
    widgets.Checkbox(value=False, description='Show Resulting HRF')
])

out = widgets.interactive_output(plot_hrf_components, 
                                 {'show_pos': ui.children[0], 
                                  'show_neg': ui.children[1], 
                                  'show_sum': ui.children[2]})

display(ui, out)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# --- We need a simple HRF generator for the code to run ---
def double_gamma_hrf(t):
    # Standard SPM parameters for HRF
    # (Simplified version of the math you saw in the modules)
    from scipy.stats import gamma
    peak = gamma.pdf(t, 6) 
    undershoot = gamma.pdf(t, 16)
    hrf = peak - 0.035 * undershoot
    return None, None, hrf / np.max(hrf) # Normalized to peak at 1

def plot_convolution(n_events=3):
    # Setup time
    tr = 0.1 
    t = np.linspace(0, 40, int(40/tr))
    
    # Create random events
    np.random.seed(42) 
    event_times = np.sort(np.random.choice(np.arange(2, 25), n_events, replace=False))
    event_amplitudes = np.random.uniform(0.8, 1.2, n_events) 
    
    # Generate the standard HRF shape
    _, _, hrf_shape = double_gamma_hrf(np.linspace(0, 32, int(32/tr)))
    
    # Prepare the plots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
    
    # Plot 1: Neural Events
    ax1.set_title(f"1. Neural Events (Stimuli)", fontsize=14)
    for onset, amp in zip(event_times, event_amplitudes):
        ax1.vlines(onset, 0, amp, colors='blue', linewidth=3)
    ax1.set_ylim(0, 1.5)
    ax1.set_ylabel("Neural Intensity")
    
    # Plot 2: Individual HRFs
    ax2.set_title("2. Individual BOLD Responses", fontsize=14)
    total_signal = np.zeros_like(t)
    
    for onset, amp in zip(event_times, event_amplitudes):
        start_idx = int(onset/tr)
        end_idx = start_idx + len(hrf_shape)
        
        if end_idx > len(t):
            clip = len(t) - start_idx
            current_hrf = hrf_shape[:clip] * amp
            total_signal[start_idx:] += current_hrf
            time_axis = t[start_idx:]
        else:
            current_hrf = hrf_shape * amp
            total_signal[start_idx:end_idx] += current_hrf
            time_axis = t[start_idx:end_idx]
            
        ax2.plot(time_axis, current_hrf, '--', alpha=0.7)
        ax2.fill_between(time_axis, current_hrf, alpha=0.1)
        
    ax2.set_ylabel("BOLD Response")
    
    # Plot 3: The Convolution
    ax3.set_title("3. Final Convolved Regressor (The Sum)", fontsize=14)
    ax3.plot(t, total_signal, 'k-', linewidth=3)
    ax3.fill_between(t, total_signal, color='gray', alpha=0.2)
    ax3.set_xlabel("Time (seconds)")
    ax3.set_ylabel("Predicted Signal")
    
    plt.tight_layout()
    plt.show()

# --- THIS IS THE FIX ---
# Define the slider
slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Events:')

# Link the function and the slider into a variable 'ui'
ui = widgets.interactive(plot_convolution, n_events=slider)

# Now display 'ui'
display(ui)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Robust imports across nilearn versions ---
try:
    # Nilearn often exposes these here
    from nilearn.glm.first_level.hemodynamic_models import (
        spm_hrf, glover_hrf,
        spm_time_derivative, glover_time_derivative,
        spm_dispersion_derivative, glover_dispersion_derivative
    )
except ImportError:
    # Fallback used in some versions
    from nilearn.glm.first_level import (
        spm_hrf, glover_hrf,
        spm_time_derivative, glover_time_derivative,
        spm_dispersion_derivative, glover_dispersion_derivative
    )

def _normalize(x):
    x = np.asarray(x).ravel()
    mx = np.max(np.abs(x))
    return x if mx == 0 else x / mx

# Parameters
t_r = 1.0
oversampling = 50
time_length = 32.0
onset = 0.0

# Build time axis consistent with nilearn's sampling
dt = t_r / oversampling
t = np.arange(0, time_length, dt)

# Compute HRFs
glover = glover_hrf(t_r=t_r, oversampling=oversampling, time_length=time_length, onset=onset)
spm = spm_hrf(t_r=t_r, oversampling=oversampling, time_length=time_length, onset=onset)

# Derivatives (basis functions)
glover_td = glover_time_derivative(t_r=t_r, oversampling=oversampling, time_length=time_length, onset=onset)
glover_dd = glover_dispersion_derivative(t_r=t_r, oversampling=oversampling, time_length=time_length, onset=onset)

spm_td = spm_time_derivative(t_r=t_r, oversampling=oversampling, time_length=time_length, onset=onset)
spm_dd = spm_dispersion_derivative(t_r=t_r, oversampling=oversampling, time_length=time_length, onset=onset)

# Plot
plt.figure(figsize=(12, 6))

plt.plot(t, _normalize(glover), linewidth=3, label="Glover (canonical)")
plt.plot(t, _normalize(spm), linewidth=3, linestyle="--", label="SPM (canonical)")

plt.plot(t, _normalize(glover_td), linestyle=":", linewidth=2, label="Glover: temporal derivative")
plt.plot(t, _normalize(glover_dd), linestyle=":", linewidth=2, label="Glover: dispersion derivative")

plt.plot(t, _normalize(spm_td), linestyle="-.", linewidth=2, label="SPM: temporal derivative")
plt.plot(t, _normalize(spm_dd), linestyle="-.", linewidth=2, label="SPM: dispersion derivative")

plt.axhline(0, linewidth=1)
plt.grid(True, alpha=0.3)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (normalized)")
plt.title("Comparing Nilearn HRF models and derivative basis functions")
plt.legend(loc="best")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Setup the variables exactly like your experiment
N_scans = 453   # Total rows in your matrix
k = 1           # Let's recreate 'drift_1'

# 2. Create the Time Indices (t = 0, 1, 2 ... 452)
t = np.arange(N_scans)

# 3. Apply the DCT Formula manually
# Formula: sqrt(2/N) * cos( pi * k * (2t + 1) / 2N )
normalization_factor = np.sqrt(2 / N_scans)
cosine_wave = np.cos((np.pi * k * (2 * t + 1)) / (2 * N_scans))

drift_1_calculated = normalization_factor * cosine_wave

# 4. Check the specific values you saw in your data
print(f"--- The Math Check ---")
print(f"Normalization Factor (sqrt(2/453)): {normalization_factor:.6f}")
print(f"\nCalculated Row 1 (t=0): {drift_1_calculated[0]:.6f}")
print(f"Calculated Row 2 (t=1): {drift_1_calculated[1]:.6f}")
print(f"Calculated Row 3 (t=2): {drift_1_calculated[2]:.6f}")

print(f"\nYour Data Row 1:        0.066445")
print("(They should match perfectly!)")

# 5. Visualizing the Mathematical Waves
plt.figure(figsize=(10, 6))

# Plot Drift 1 (The Slowest Wave)
plt.plot(drift_1_calculated, label='Calculated Drift 1', linewidth=3, color='blue')

# Plot Drift 3 (A Faster Wave)
k3 = 3
drift_3_calculated = normalization_factor * np.cos((np.pi * k3 * (2 * t + 1)) / (2 * N_scans))
plt.plot(drift_3_calculated, label='Calculated Drift 3', linewidth=2, color='orange', linestyle='--')

# Plot Drift 9 (The Fastest Wave)
k9 = 9
drift_9_calculated = normalization_factor * np.cos((np.pi * k9 * (2 * t + 1)) / (2 * N_scans))
plt.plot(drift_9_calculated, label='Calculated Drift 9', linewidth=1, color='green', alpha=0.7)

plt.title(f"The Math Behind the Drift Columns (N={N_scans})")
plt.xlabel("Scan Index (Time)")
plt.ylabel("Value (Amplitude)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()